In [ ]:
# prompt: from kaggle json download kaggle dataset and unzip

!pip install kaggle

# Upload your kaggle.json file (contains API key)
# from google.colab import files
# files.upload()


{}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#/content/drive/MyDrive/LearnAboutTfrecord/First/kaggle.json

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/LearnAboutTfrecord/First/kaggle.json ~/.kaggle/

In [ ]:

# Make directory for kaggle and move the json file there

# !cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Replace with your desired dataset




In [ ]:

# Replace with your desired dataset
!kaggle datasets download -d austinreese/craigslist-carstrucks-data

Dataset URL: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data
License(s): CC0-1.0
 96% 252M/262M [00:01<00:00, 202MB/s]
100% 262M/262M [00:01<00:00, 209MB/s]


In [ ]:
# unzip the downloaded dataset
!unzip /content/craigslist-carstrucks-data.zip

Archive:  /content/craigslist-carstrucks-data.zip
  inflating: vehicles.csv            


In [ ]:
import os
import  tensorflow as tf
import pandas as pd
import numpy as np


In [ ]:
data=pd.read_csv('/content/vehicles.csv')
data.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [ ]:
data.shape

(426880, 26)

In [ ]:
data

,0
id,0
url,0
region,0
region_url,0
price,0
year,1205
manufacturer,17646
model,5277
condition,174104
cylinders,177678


In [ ]:
data=data.drop(['county', 'size','paint_color','drive','VIN','condition','cylinders'], axis=1)

In [ ]:
data.isnull().sum()

,0
id,0
url,0
region,0
region_url,0
price,0
year,1205
manufacturer,17646
model,5277
fuel,3013
odometer,4400


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isnull().sum()

,0
id,0
url,0
region,0
region_url,0
price,0
year,0
manufacturer,0
model,0
fuel,0
odometer,0


In [ ]:
data=data[["region",'price','year','fuel','odometer',"lat",'type']]

In [ ]:
for i in data.dtypes:
  print(i)

object
int64
float64
object
float64
float64
object


In [ ]:
def _int64_feature(value):
  if type(value) == list:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
  else:
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
  if type(value) == list:
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  else:
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _bytes_feature(value):
  if type(value)==list:

    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
  else:
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


_int64_feature(1)
_int64_feature([1,2,3])
_float_feature(2.0)
_float_feature([1.0,2.0,3.0])




float_list {
  value: 1
  value: 2
  value: 3
}

In [ ]:


def _bytes_feature(value):
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  if isinstance(value, list):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
  elif isinstance(value, bytes):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
  else:
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))

In [ ]:
intfeature=_int64_feature([1,2,3])
ffeature=_float_feature([1.0,2.0])
bytefeature=_bytes_feature([b"this is a string",b"this is another string"])

ffeature
bytefeature

bytes_list {
  value: "this is a string"
  value: "this is another string"
}

In [ ]:

fs = tf.train.Features(feature={'intlist': intfeature, 'floatlist': ffeature,"bytefeature":bytefeature})
fs

tf.train.Example(features=fs)

features {
  feature {
    key: "intlist"
    value {
      int64_list {
        value: 1
        value: 2
        value: 3
      }
    }
  }
  feature {
    key: "floatlist"
    value {
      float_list {
        value: 1
        value: 2
      }
    }
  }
  feature {
    key: "bytefeature"
    value {
      bytes_list {
        value: "this is a string"
        value: "this is another string"
      }
    }
  }
}

In [ ]:


def get_feature_description(types):
  feature_description = {}
  for key, value in types.items():
    if value == 'str' or value == 'bytes':
      feature_description[key] = tf.io.FixedLenFeature([], tf.string)
    elif value == 'int':
      feature_description[key] = tf.io.FixedLenFeature([], tf.int64)
    elif value == 'float':
      feature_description[key] = tf.io.FixedLenFeature([], tf.float32)
  return feature_description

In [ ]:
# prompt: types={}
# typee=list(data.dtypes)
# for i,j  in enumerate(data.dtypes):
#   types[data.columns[i]]=i

types = {}
typee = list(data.dtypes)
for i, j in enumerate(data.dtypes):
  if j == 'object':
    types[data.columns[i]] = 'str'
  elif j == 'int64':
    types[data.columns[i]] = 'int'
  elif j == 'float64':
    types[data.columns[i]] = 'float'
  else:
    types[data.columns[i]] = str(j)
types

{'region': 'str',
 'price': 'int',
 'year': 'float',
 'fuel': 'str',
 'odometer': 'float',
 'lat': 'float',
 'type': 'str'}

In [ ]:
data.dtypes

,0
region,object
price,int64
year,float64
fuel,object
odometer,float64
lat,float64
type,object


In [ ]:
#types = {'Car':'str','MPG':'float','Cylinders':'int','Displacement':'float','Horsepower':'float','Weight':'float','Acceleration':'float','Model':'int'}

In [ ]:
values_list = list(types.values())
keys_list = list(types.keys())

In [ ]:
values_list

['str', 'int', 'float', 'str', 'float', 'float', 'str']

In [ ]:
keys_list

['region', 'price', 'year', 'fuel', 'odometer', 'lat', 'type']

In [ ]:


import tensorflow as tf

def convert_toexample(row, column_types_list, column_names_list):
  feature_dict = {}
  for i in range(0,len(row)):
    if column_types_list[i] == 'str' or column_types_list[i] == 'byte':
      feature_dict[column_names_list[i]] = _bytes_feature(str.encode(str(row[i]))) # Encode to bytes
    elif column_types_list[i] == 'int':
      feature_dict[column_names_list[i]] = _int64_feature(int(row[i])) # Convert to int
    elif column_types_list[i] == 'float':
      feature_dict[column_names_list[i]] = _float_feature(float(row[i])) # Convert to float
  example = tf.train.Example(features=tf.train.Features(feature = feature_dict))
  return example

# def _int64_feature(value):
#   return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# def _float_feature(value):
#   return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# def _bytes_feature(value):
#   if isinstance(value, type(tf.constant(0))):
#     value = value.numpy()
#   return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [1]:
with tf.io.TFRecordWriter(path="cars.tfrecords") as writer:
  for _, row in data.iterrows():
    example = convert_toexample(row=row, column_types_list=values_list, column_names_list=keys_list)
    writer.write(example.SerializeToString())

NameError: name 'tf' is not defined

In [ ]:
data.to_csv('cars.csv')

In [ ]:
# prompt: def parse_example(record):
#   return tf.io.parse_single_example(record, get_feature_description())
# cars_dataset = tf.data.TFRecordDataset("cars.tfrecords")
# parsed_tfrecords = cars_dataset.map(map_func=parse_example)
# for tfrecord in parsed_tfrecords:
#   print(tfrecord['Car'].numpy())

def get_feature_description():
  feature_description = {
      'region': tf.io.FixedLenFeature([], tf.string),
      'price': tf.io.FixedLenFeature([], tf.float32),
      'year': tf.io.FixedLenFeature([], tf.int64),
      'fuel': tf.io.FixedLenFeature([], tf.string),
      'odometer': tf.io.FixedLenFeature([], tf.float32),
      'lat': tf.io.FixedLenFeature([], tf.float32),
      'type': tf.io.FixedLenFeature([], tf.string)
  }
  return feature_description

def parse_example(record):
  return tf.io.parse_single_example(record, get_feature_description())

cars_dataset = tf.data.TFRecordDataset("cars.tfrecords")
parsed_tfrecords = cars_dataset.map(map_func=parse_example)

for tfrecord in parsed_tfrecords:
  print(tfrecord['region'].numpy())

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_7_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:6 transformation with iterator: Iterator::Root::ParallelMapV2: Key: year.  Data types don't match. Data type: float but expected type: int64
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 